In [ ]:
import PySDM.products as PySDM_products
from PySDM import Builder
from PySDM import Formulae
from Loftus_and_Wordsworth_2021.settings import Settings, EARTH_LIKE_CONSTS
from Loftus_and_Wordsworth_2021.simulation import Simulation
import matplotlib.pyplot as plt

In [ ]:
#Have to add the following equations that are in the paper to the formulae section:
#Will go under PySDM/physics/ wherever these categories are
formulae= Formulae(
          terminal_velocity=..., #eqn 8
          drop_growth=..., #eqn 10
          ventilation=..., #drag force/ gravitation eqns
          particle_shape_and_density=..., #the following OblateSpheroid class
)

#For particle_shape and density, we will need to create a new class for liquid oblate spheroids
#under PySDM/particle_shape_and_density.py
#this needs to include eqn 2 about the equivalent radius, Reynolds number eqn (5)... 
#look at LiquidSphere class for reference
class OblateSpheroid:
    def __init__(self, a, b):
        self.a = a  
        self.b = b

    def volume(self):
        return 
    



In [ ]:
#settings in this folder, some inputs are from the paper needed for the plot, like initial radius
settings = Settings(        
        updraft_velocity=0, # varied in the paper, add units
        r_dry= 1e-16, #is this in the paper?
        r_wet= initial_radius, #X axis of plot 2
        formulae= formulae,
        Planet= EARTH_LIKE_CONSTS,)

In [ ]:
#set up simulation
#Simulation is also in this folder
#Need to change/add/edit the Parcel environment PySDM/parcel to take in new dz that uses the 
#updraft and the terminal velocity to have the parcel track the superdroplet
simulation = Simulation(settings,..)
output = simulation.run()

#output currently has height, RH, radius, and time.. need others?
plt.plot(output['time'], output['radius'])
